# Compressed Air Production

Story

- Wo steckt bei Druckluft die Energie? Energiebilanz über Kompression und wieder Abkühlen ergibt ~0
- Man kann isentrope Arbeit mit Exergie gleichsetzen
- Man kann der Abkühlung einen thermischen Exergieanteil zuordnen, mechanische Exergie (sollte) sich nicht ändern

Zwischenübung

- Drücke/variieren, ...

Story 2

- Man kann die einfache Abkühlung auch mit einem WÜT abbilden -> Jetzt wirds kompliziert, weil 2 Komponenten
- Analyse manuell
- Nutze lieber eine Framework, TESPy und das geht so: ...

Übung

- Gleiche Analyse mit TESPy, Vergleich mit Ergebnissen von vorher

## Introduction

We want to consider compression of air and cooling it the fluid to the ambient
temperature.

```{math}
    :label: eq:compressed-air-information
    T_3 = T_1\\
    p_3 > p_1
```

Applying energy analysis to the overall system neglecting the influence of
potential and kinetic energy to the thermal system, we can calculate the total
amount of energy transferred over the system boundaries $\dot E$:

```{math}
    :label: eq:compressed-air-energy
    \dot E =\dot m \cdot \left( h_3 - h_1\right)
```

However, we can see that the change in enthalpy is nearly zero, since $h_1$ and
$h_3$ are almost equal.

```{math}
    :label: eq:compressed-air-enthalpy-equality
    h_\left(p_1, T_1\right) \sim h_\left(p_3, T_3\right)
```

We can easily verify this claim:

In [ ]:
from CoolProp.CoolProp import PropsSI

T_amb = 30  # °C
p_amb = 1.01325  # bar
fluid = "air"

p_3 = 5  # bar

p_amb_SI = p_amb * 1e5  # Pa
p_3_SI = p_3 * 1e5  # Pa
T_amb_SI = T_amb + 273.15  # K

h_1 = PropsSI("H", "P", p_amb_SI, "T", T_amb_SI, fluid)
h_3 = PropsSI("H", "P", p_3_SI, "T", T_amb_SI, fluid)

print(f"Change in enthalpy: {round(h_3 - h_1)} J/kg")

It even looks as if energy has been produced by our setup, since the change in enthalpy is
negative. We can see, how the enthalpy of air changes with pressure given 
isothermal process:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

p_ref = 1e3  # Pa
p_max = 1e6  # Pa
p_range = np.linspace(p_ref, p_max)  # Pa
T_ref = 303.15  # K

h_ref = PropsSI("H", "P", p_ref, "T", T_ref, fluid)
h = PropsSI("H", "P", p_range, "T", T_ref, fluid)

fig, ax = plt.subplots(1)

ax.plot(p_range, h - h_ref)
ax.set_xlabel("$\Delta p$ in $Pa$")
ax.set_ylabel("$\Delta h$ in $J/kg$")

ax.set_axisbelow(True)
ax.grid(color='gray', linestyle='dashed')

plt.tight_layout()

For ideal gases, the change in enthalpy will be exactly zero, as the enthalpy
does not depend on change in pressure but only on change in temperature:

```{math}
    :label: eq:compressed-air-enthalpy-ideal-gas
    h\left(T\right) - h\left(T_\text{ref}\right) = c_\text{p}\left(T\right) \cdot \left(T - T_\text{ref}\right)
```

```{math}
    :label: eq:compressed-air-enthalpy-equality2
    h_\left(p_1, T_1\right) \sim h_\left(p_3, T_3\right)
```

To visualize the energy stored in the pressurized air, we can apply the exergy
method and split the exergy in its thermal and mechanical shares.

In [ ]:
from tespy.connections import Connection
from tespy.components import Sink, Source
from tespy.networks import Network


Network(["air"])


ref = Connection(Source("Dummysource1"), "out1", Sink("Dummysink1"), "in1")
c = Connection(Source("Dummysource2"), "out1", Sink("Dummysink2"), "in1")

ref.fluid.val = {"air": 1}
ref.p.val_SI = p_ref
ref.T.val_SI = T_ref
ref.h.val_SI = PropsSI("H", "P", p_ref, "T", T_ref, "air")
ref.s.val_SI = PropsSI("S", "P", p_ref, "T", T_ref, "air")

ref.get_physical_exergy(p_ref, T_ref)
print(f"Mechanical exergy at reference state: {round(ref.ex_mech)} J/kg")
# print(f"Thermal exergy at reference state: {round(ref.ex_therm)} J/kg")

c.fluid.val = {"air": 1}
c.T.val_SI = T_ref

ex = []

for p in p_range:
    c.p.val_SI = p
    c.h.val_SI = PropsSI("H", "P", p, "T", T_ref, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T_ref, "air")

    c.get_physical_exergy(p_ref, T_ref)
    ex += [c.ex_mech]
    

ax.plot(p_range, ex)    
display(fig)
plt.close(fig)

With a closer look at the compressor assuming isentropic compression, we see that
the physical exergy $ex^\text{PH}$ of the compressed air is identical to the 
isentropic compression energy $w = h_\text{is} - h$:

In [ ]:
s_ref = PropsSI("S", "P", p_ref, "T", T_ref, "air")
h_is = PropsSI("H", "P", p_range, "S", s_ref, "air")
T_is = PropsSI("T", "P", p_range, "S", s_ref, "air")

ex_therm = []
ex_mech = []
ex = []

for p, T in zip(p_range, T_is):
    c.h.val_SI = PropsSI("H", "P", p, "T", T, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T, "air")
    c.p.val_SI = p
    c.get_physical_exergy(p_ref, T_ref)
    ex_therm += [c.ex_therm]
    ex_mech += [c.ex_mech]
    ex += [c.ex_physical]
    

fig, ax = plt.subplots(1)

ax.set_xlabel("$\Delta p$ in $Pa$")
ax.set_ylabel("$\Delta h$ in $J/kg$")

ax.set_axisbelow(True)
ax.grid(color='gray', linestyle='dashed')

plt.tight_layout()
    
ax.plot(p_range, h_is - ref.h.val_SI, "x-", label="$w_{is}$")
ax.plot(p_range, ex_mech, label="$ex^{M}_{is}$")
ax.plot(p_range, ex_therm, label="$ex^{T}_{is}$")
ax.plot(p_range, ex, "--", label="$ex_{PH,is}$")

ax.legend()
plt.show()

We can now inspect the change of energy in the cooling process (assuming isobaric).

In [ ]:
ex_12 = ex

h_is = PropsSI("H", "P", p_range, "S", s_ref, "air")
h_3 = PropsSI("H", "P", p_range, "T", T_ref, "air")

dex_therm = []
dex_mech = []
dex = []

for p, T in zip(p_range, T_is):
    c.h.val_SI = PropsSI("H", "P", p, "T", T, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T, "air")
    c.p.val_SI = p
    c.get_physical_exergy(p_ref, T_ref)
    ex_therm_is, ex_mech_is, ex_is = c.ex_therm, c.ex_mech, c.ex_physical
    c.h.val_SI = PropsSI("H", "P", p, "T", T_ref, "air")
    c.s.val_SI = PropsSI("S", "P", p, "T", T_ref, "air")
    c.p.val_SI = p
    c.get_physical_exergy(p_ref, T_ref)
    dex_therm += [c.ex_therm - ex_therm_is]
    dex_mech += [c.ex_mech - ex_mech_is]
    dex += [c.ex_physical - ex_is]
    

fig, ax = plt.subplots(1)

ax.set_xlabel("$\Delta p$ in $Pa$")
ax.set_ylabel("$\Delta h$ in $J/kg$")

ax.set_axisbelow(True)
ax.grid(color='gray', linestyle='dashed')

plt.tight_layout()

ax.plot(p_range, h_3 - h_is, "x-", label="$q_{2,3}$")
ax.plot(p_range, dex_therm, "o", label="$\Delta ex^{T}_{2,3}$")
ax.plot(p_range, dex_mech, label="$\Delta ex^{M}_{2,3}$")
ax.plot(p_range, dex, label="$\Delta ex^{PH}_{2,3}$")
ax.plot(p_range, ex, label="$\Delta ex^{PH}_{1,2}$")
ax.plot(p_range, [ex[i] + dex[i] for i in range(len(ex))], label="$\Delta ex^{PH}_{1,3}$")

ax.legend()
plt.show()

- Nur Kompressor vs. Kompressor mit Zwischenkühler:
  - Was ist mit dem MItteldruck
  - Nutzen der Zwischenkühlung?
  - Wem ist die Zwischenkühlung im Vergleich zuzuordnen?

- T,s/h,s-Diagramm
- Bezug Integral v dp?